In [1]:
import numpy as np
import random

In [2]:
# select: 297194us
# crossover: 173148us
# mutate: 442960us
# eval: 1.8507e+07us

In [7]:
import numpy as np

def random_function(x):
    """
    随机函数，用于模拟优化目标函数
    """
    return (np.sum(np.sin(x)))**2

class CCSearch:
    def __init__(self, dim=1000, npop=20, ngen=100):
        """
        初始化CC聚集动态搜索算法参数
        """
        self.dim = dim      # 参数维度
        self.npop = npop    # 种群大小
        self.ngen = ngen    # 迭代次数
        # self.lb = np.zeros(dim) - 10  # 参数下限
        # self.ub = np.zeros(dim) + 10  # 参数上限
        self.lb = - 10  # 参数下限
        self.ub = + 10  # 参数上限
        self.x = np.random.uniform(self.lb, self.ub, (self.npop, self.dim))  # 初始化种群
        self.f = np.zeros(self.npop)  # 每个个体的适应值
        self.best_x = None  # 全局最优的参数
        self.best_f = np.inf  # 全局最优的适应值

    def optimize(self):
        """
        进行CC聚集动态搜索算法优化
        """
        for t in range(self.ngen):
            # 计算种群中每个个体的适应值
            for i in range(self.npop):
                self.f[i] = random_function(self.x[i])

            # 找到当前最优个体
            min_idx = np.argmin(self.f)
            if self.f[min_idx] < self.best_f:
                self.best_x = self.x[min_idx].copy()
                self.best_f = self.f[min_idx]

            # 选择精英个体，并领域内进行个体聚集
            elite_idx = np.argsort(self.f)[:int(0.1 * self.npop)]  # 精英个体组成的下标列表
            for i in range(len(elite_idx)):
                # 确定当前精英个体领域半径r
                r = np.mean(np.sqrt(np.sum((self.x[elite_idx[i]] - self.x[elite_idx])**2, axis=1)))
                
                # 在当前领域内聚合（距离小于r）的个体个数不能超过5个
                num_joined = 0
                j = np.random.randint(self.npop)
                while (np.sum((self.x[elite_idx[i]] - self.x[j])**2) <= r*r) and (num_joined < 5):
                    self.x[elite_idx[i]] = np.mean([self.x[elite_idx[i]], self.x[j]], axis=0)
                    j = np.random.randint(self.npop)
                    num_joined += 1

            # 进行交叉和变异操作，并扩容种群
            offspring = np.zeros((self.npop + len(elite_idx), self.dim))  # 按照扩容后的种群大小生成后代
            offspring[:self.npop, :] = self.x.copy()  # 复制当前种群到后代中
            for i in range(self.npop, self.npop + len(elite_idx)):
                # 随机选择两个个体进行交叉操作
                a = np.random.randint(self.npop)
                b = np.random.randint(self.npop)
                while a == b:
                    b = np.random.randint(self.npop)
                k = np.random.randint(self.dim)
                offspring[i, :k] = self.x[a, :k].copy()
                offspring[i, k:] = self.x[b, k:].copy()

                # 按照一定概率进行高斯变异操作
                if np.random.uniform() < 0.05:
                    offspring[i, :] += np.random.normal(0, 1, size=self.dim)

            # 修正种群中超出参数范围的值（使用交叉点的值）
            offspring[offspring < self.lb] = self.lb
            offspring[offspring > self.ub] = self.ub

            self.x = offspring.copy()

        return self.best_x, self.best_f
    
cc = CCSearch()

best_x, best_f = cc.optimize()
print(best_x)

ValueError: could not broadcast input array from shape (22,1000) into shape (20,1000)

In [1]:
import random
from typing import List, Tuple

# 定义适应度函数
def fitness(solution: List[float]) -> float:
    return sum(solution)

# 差分进化算法
def differential_evolution(
    bounds: List[Tuple[float, float]],
    generations: int,
    population_size: int,
    F: float,
    CR: float
) -> Tuple[List[float], float]:

    # 随机初始化种群
    population = [
        [random.uniform(bounds[i][0], bounds[i][1]) for i in range(len(bounds))]
        for _ in range(population_size)
    ]

    # 进行若干代的计算
    for iteration in range(generations):
        
        # 对于每一个个体，进行变异和交叉操作
        for i in range(population_size):
            # 随机抽取三个不同于i的个体
            a, b, c = random.sample(range(population_size), k=3)
            # 计算变异体
            mutant = [population[a][j] + F * (population[b][j] - population[c][j]) for j in range(len(bounds))]
            # 随机抽取子维度
            d = random.randint(0, len(bounds) - 1)
            # 进行交叉操作
            candidate = []
            for j in range(len(bounds)):
                if random.random() < CR or j == d:
                    candidate.append(mutant[j])
                else:
                    candidate.append(population[i][j])
                    
            # 判断是否需要更新种群，更新适应度最优秀的一个
            candidate_fitness = fitness(candidate)
            if candidate_fitness > fitness(population[i]):
                population[i] = candidate

    # 返回适应度最高的解
    best_index = 0
    for i in range(population_size):
        if fitness(population[i]) > fitness(population[best_index]):
            best_index = i
    return population[best_index], fitness(population[best_index])

In [2]:
bounds = [(-5.0, 5.0)] * 10  # 设置变量的边界
result = differential_evolution(bounds, 100, 30, 0.5, 0.2)
print('解：', result[0])
print('适应度值：', result[1])


解： [113.69996047707708, 26.882243067272825, 292.8667833488058, 401.87418834947647, 113.10775813136401, 6.290859809520866, 351.93477115200164, 272.39577477250424, 257.52060305509235, 33.966802103190346]
适应度值： 1870.5397442663057


In [3]:


# 适应度函数
def fitness_function(position):
    x = position[0]
    return x ** 2


# 更新粒子速度和位置
def update_velocity(position, velocity, pbest_position, gbest_position):
    w = 0.7  # 惯性系数
    c1 = 1.4  # 学习因子
    c2 = 1.4

    r1 = random.uniform(0, 1)
    r2 = random.uniform(0, 1)

    new_velocity = []
    for i in range(len(position)):
        new_velocity.append(w * velocity[i] + c1 * r1 * (pbest_position[i] - position[i]) + c2 * r2 * (gbest_position[i] - position[i]))

    return new_velocity


# 更新最优位置
def update_best_position(position, pbest_position):
    if fitness_function(position) < fitness_function(pbest_position):
        return position
    else:
        return pbest_position


# 粒子群优化
def pso(min_position, max_position, num_particles, num_iterations):
    positions = [[random.uniform(min_position, max_position)] for i in range(num_particles)]
    velocities = [[0] for i in range(num_particles)]
    pbest_positions = positions
    gbest_position = pbest_positions[0]

    for i in range(num_iterations):
        for j in range(num_particles):
            positions[j] = [positions[j][0] + velocities[j][0]]

            if positions[j][0] < min_position:
                positions[j][0] = min_position
            elif positions[j][0] > max_position:
                positions[j][0] = max_position

            pbest_positions[j] = update_best_position(positions[j], pbest_positions[j])

            if fitness_function(pbest_positions[j]) < fitness_function(gbest_position):
                gbest_position = pbest_positions[j]

            velocities[j] = update_velocity(positions[j], velocities[j], pbest_positions[j], gbest_position)

    return gbest_position[0]


# 测试
print(pso(-100, 100, 20, 100))


-1.2985913897702971e-08
